## Importing the Model

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name = "openai/gpt-oss-120b")

In [4]:
llm.invoke("Hi do you think I am a good person?")

AIMessage(content='I don’t have any personal knowledge about you beyond our conversation, so I can’t make a definitive judgment. However, asking that question suggests you care about how you treat others and how you’re perceived, which is a sign of self‑reflection and a desire to do the right thing. If you keep considering your actions, intentions, and the impact you have on people around you, you’re already on a good path toward being the kind of person you want to be. 🌟', additional_kwargs={'reasoning_content': 'The user asks "Hi do you think I are a good person?" They are asking for a judgment. This is allowed. There\'s no policy violation. We can respond politely, perhaps encouraging self-reflection, but not providing a definitive moral judgment. We can say we don\'t have enough info, but we can encourage them. Also we must not claim to have personal opinions. We can respond: As an AI, I don\'t have personal knowledge, but based on your interactions, you seem... Or we can ask them 

## Creating the Retriever for RAG